In [14]:
# !pip install joblib
# !pip install gensim
# !pip install glove-python-binary
# !pip install transformers
# !pip install datasets
# !pip install tokenizers

In [1]:
import time
import os
import psutil

def count_time(func):
    def int_time():
        start_time = time.time()
        func()
        over_time = time.time()
        total_time = over_time - start_time
        print("程序运行了%s秒" % total_time)
    return int_time

def count_info(func):
    def float_info():
        pid = os.getpid()
        p = psutil.Process(pid)
        info_start = p.memory_full_info().uss/1024
        func()
        info_end=p.memory_full_info().uss/1024
        print("程序占用了内存"+str(info_end-info_start)+"KB")
    return float_info

In [2]:
import numpy as np
import gc
import pandas as pd
import re
from joblib import Parallel, delayed
from gensim.models import FastText,Word2Vec
from glove import Glove,Corpus
from sklearn.model_selection import KFold
from sklearn.cluster import KMeans
from scipy import sparse
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import (AutoModel,AutoModelForMaskedLM, 
                          AutoTokenizer, LineByLineTextDataset,
                          DataCollatorForLanguageModeling,
                          Trainer, TrainingArguments,PreTrainedTokenizerFast,pipeline)
from datasets import Dataset
from sklearn.linear_model import Ridge
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)
import warnings
warnings.filterwarnings('ignore')

from tqdm.auto import tqdm

In [4]:
from NLP_feature import NLP_feature

In [5]:
df_train = pd.read_csv('sub_train.csv')
df_test = pd.read_csv('sub_val.csv')

In [6]:
df_test.head(3)

,index,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,...,release_date,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew,revenue
0,2,3,NaN,3300000,"[{'id': 18, 'name': 'Drama'}]",http://sonyclassics.com/whiplash/,tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",...,10/10/14,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The road to greatness can take you to the edge.,Whiplash,"[{'id': 1416, 'name': 'jazz'}, {'id': 1523, 'n...","[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",13092000
1,3,4,NaN,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",http://kahaanithefilm.com/,tt1821480,hi,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,...,3/9/12,122.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Kahaani,"[{'id': 10092, 'name': 'mystery'}, {'id': 1054...","[{'cast_id': 1, 'character': 'Vidya Bagchi', '...","[{'credit_id': '52fe48779251416c9108d6eb', 'de...",16000000
2,4,5,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,tt1380152,ko,마린보이,Marine Boy is the story of a former national s...,...,2/5/09,118.0,"[{'iso_639_1': 'ko', 'name': '한국어/조선말'}]",Released,NaN,Marine Boy,NaN,"[{'cast_id': 3, 'character': 'Chun-soo', 'cred...","[{'credit_id': '52fe464b9251416c75073b43', 'de...",3923970


In [7]:
use_Toknizer=True
emb_mode = 'Word2Vec'
encode_mode = 'supervise'
text_columns_name = ['overview']
target_column = df_train['revenue']
candidate_labels=None

In [8]:
nlp = NLP_feature()
# nlp.tokenizers
nlp.do_mlm = True
nlp.emb_size=100
nlp.n_clusters=20
@count_time
@count_info
def fit():
    return nlp.fit(df_train,
             text_columns_name,
             use_Toknizer,
             emb_mode,
             encode_mode,
             target_column,
             candidate_labels)
df = fit()

Fitting column: overview tokenizer



Tokenizing column: overview
Fitting column: overview word2vec embedding
Fitting column: overview encoder
程序占用了内存60784.0KB
程序运行了4.327150583267212秒


In [9]:
# for column in df.columns:
#     df_train[column] = df[column]

In [10]:
@count_time
@count_info
def trans():
    return nlp.transform(df_test)
test = trans()

Transforming column: item_description
Tokenizing column: item_description
程序占用了内存-3488.0KB
程序运行了1.3236894607543945秒


In [12]:
# df_train.head(1)

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,category_name_meta_feature,item_description_meta_feature,name_meta_feature
0,39376,Size 8 Boots,2,Women/Shoes/Boots,Sonoma,24.0,0,Excellent condition! Worn maybe once or twice,18.0,19.0,16.0


In [13]:
# df_train.to_csv(f'mlm_{emb_mode}_{encode_mode}_autox_trn.csv',index=False)
# test.to_csv(f'mlm_{emb_mode}_{encode_mode}_autox_tst.csv',index=False)